## 📦 Setup & Installation

In [1]:
!mkdir Data
!unzip music-classifier-v2.2-colab.zip

Archive:  music-classifier-v2.2-colab.zip
   creating: music-genre-classifier/
  inflating: music-genre-classifier/requirements.txt  
  inflating: music-genre-classifier/predict.py  
   creating: music-genre-classifier/utils/
  inflating: music-genre-classifier/utils/metrics.py  
  inflating: music-genre-classifier/utils/augmentation.py  
  inflating: music-genre-classifier/utils/__init__.py  
  inflating: music-genre-classifier/utils/audio_processor.py  
  inflating: music-genre-classifier/utils/dataset.py  
  inflating: music-genre-classifier/utils/losses.py  
   creating: music-genre-classifier/models/
  inflating: music-genre-classifier/models/__init__.py  
  inflating: music-genre-classifier/models/cnn_model.py  
  inflating: music-genre-classifier/Music_Genre_Classification_Complete.ipynb  
  inflating: music-genre-classifier/README.md  
  inflating: music-genre-classifier/train.py  
  inflating: music-genre-classifier/COLAB_QUICKSTART.md  
  inflating: music-genre-classifier/eva

In [2]:
# Install required packages
!pip install -q torch torchaudio librosa soundfile matplotlib seaborn scikit-learn tqdm

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchaudio
import librosa

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Using device: cuda
GPU: Tesla T4
Memory: 15.83 GB


## 📁 Upload Data

อัปโหลด `Data.zip` หรือโฟลเดอร์ `Data/` ที่มีโครงสร้าง:
```
Data/
└── genres_original/
    ├── blues/
    ├── classical/
    ├── country/
    ...
```

In [4]:
# ถ้าอัปโหลด Data.zip
!unzip Data.zip
# หรือ mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/your-path/Data .

Archive:  Data.zip
  inflating: features_30_sec.csv     
  inflating: features_3_sec.csv      
  inflating: genres_original/blues/blues.00000.wav  
  inflating: genres_original/blues/blues.00001.wav  
  inflating: genres_original/blues/blues.00002.wav  
  inflating: genres_original/blues/blues.00003.wav  
  inflating: genres_original/blues/blues.00004.wav  
  inflating: genres_original/blues/blues.00005.wav  
  inflating: genres_original/blues/blues.00006.wav  
  inflating: genres_original/blues/blues.00007.wav  
  inflating: genres_original/blues/blues.00008.wav  
  inflating: genres_original/blues/blues.00009.wav  
  inflating: genres_original/blues/blues.00010.wav  
  inflating: genres_original/blues/blues.00011.wav  
  inflating: genres_original/blues/blues.00012.wav  
  inflating: genres_original/blues/blues.00013.wav  
  inflating: genres_original/blues/blues.00014.wav  
  inflating: genres_original/blues/blues.00015.wav  
  inflating: genres_original/blues/blues.00016.wav  
  in

In [5]:
!cp -r genres_original/ Data
!cp -r images_original/ Data
!cp features_30_sec.csv Data
!cp features_3_sec.csv Data

In [6]:
!rm Data/genres_original/jazz/jazz.00054.wav

In [11]:
!python music-genre-classifier/train.py \
      --data_dir /content/Data/genres_original \
      --model ImprovedMusicCNN \
      --epochs 50 \
      --batch_size 64 \
      --lr 0.001 \
      --use_class_weights \
      --use_focal_loss \
      --augmentation specaugment \
      --augmentation_prob 0.5 \
      --num_workers 2 \
      --save_dir /content/checkpoints \
      --early_stopping 99

Using device: cuda

Initializing audio processor...
Using SpecAugment (p=0.5)

Loading dataset from /content/Data/genres_original...
Loaded 999 audio files from 10 genres
Genres: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
Dataset split:
  Train: 699 samples (70.0%)
  Val:   149 samples (15.0%)
  Test:  151 samples (15.0%)
✓ Augmentation enabled for training set only

Creating model: ImprovedMusicCNN
Model parameters: 4,962,186

Setting up loss function...
Using Weighted Focal Loss
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioD

In [12]:
!python music-genre-classifier/evaluate.py \
    --checkpoint /content/checkpoints/best_model.pth \
    --data_dir /content/Data/genres_original

Using device: cuda

Loading checkpoint from /content/checkpoints/best_model.pth...

Initializing audio processor...

Loading dataset from /content/Data/genres_original...
Loaded 999 audio files from 10 genres
Genres: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

Creating model: ImprovedMusicCNN

Evaluating model...
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcod